In [7]:
from scipy.stats import norm
import numpy as np
import math

In [4]:
# 1. given alpha, calculte z-score
def z_score(a):
    return -1*norm.ppf(a/2)

In [98]:
# 2. given alpha, sample size, calculate beta
def return_beta(z, p1, d_min, N):
    c = p1 + z*math.sqrt(p1*(1-p1)*2/N)
    p2 = p1 + d_min
    se = math.sqrt(p2*(1-p2)*2/N)
    return norm.cdf(c, loc = p2, scale = se)

In [99]:
# for a series of N, find smallest N that will give target beta
def sample_size(alpha, beta, p1, d_min):
    z = z_score(alpha)
    n = 1
    T = True
    while T:
        if return_beta(z, p1, d_min, n) <= beta:
            T = False
        n +=1
    return n-1

In [100]:
# firstly try if we need to use Bonferroni correction
alpha_total = 0.05
b = 0.2
n1= sample_size(0.05, b, 0.53, 0.01)
n2= sample_size(0.05, b, 0.11, 0.0075)

In [101]:
n1

39071

In [102]:
n2

27802

# Sanity Check

## Number of Cookies

In [138]:
def confidence_interval(n_con, n_exp, p, a):
    p_test = n_con/(n_con + n_exp)
    se = math.sqrt(p*(1-p)/(n_con + n_exp))
    z = z_score(a)
    # margin of error
    m = z * se
    return p - m, p + m, p_test, ((p_test >= p-m) & (p_test <= p + m))

In [139]:
n_con = 345543
n_exp = 344660
p = 0.5
a = 0.05
confidence_interval(n_con, n_exp, p, a)

(0.49882041382459419, 0.50117958617540581, 0.5006396668806133, True)

## Number of Clicks

In [140]:
n_con = 28378
n_exp = 28325
p = 0.5
a = 0.05
confidence_interval(n_con, n_exp, p, a)

(0.49588457134714631, 0.50411542865285364, 0.5004673474066628, True)

## Click Through Probability

In [141]:
p_con = 28378/345543
p_exp = 28325/344660
d_min = p_exp - p_con

In [146]:
p_pool = (28378 + 28325)/(345543 + 344660)

In [148]:
se_pool = math.sqrt(p_pool * (1 - p_pool)*(1/345543 + 1/344660))
se_pool

0.0006610608156387222

In [144]:
a = 0.05
z = z_score(a)
m = z * se_pool

In [150]:
-m, m, d_min

(-0.0012956553902425685, 0.0012956553902425685, 5.662709158693602e-05)

# Check for Practical and Statistical Significance


####  - For your evaluation metrics, calculate a confidence interval for the difference between the experiment and control groups

#### - A metric is statistically significant if the confidence interval does not include 0 (that is, you can be confident there was a change)

#### - A metric is practically significant if the confidence interval does not include the practical significance boundary (that is, you can be confident there is a change that matters to the business.)

### Retention
#### - payment/enroll

In [152]:
import pandas as pd

In [154]:
df_con = pd.read_excel('/Users/shuyijiang1/Downloads/Final Project Results.xlsx', sheetname= 'Control')
df_exp = pd.read_excel('/Users/shuyijiang1/Downloads/Final Project Results.xlsx', sheetname= 'Experiment')

In [155]:
df_con.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134,70
1,"Sun, Oct 12",9102,779,147,70
2,"Mon, Oct 13",10511,909,167,95
3,"Tue, Oct 14",9871,836,156,105
4,"Wed, Oct 15",10014,837,163,64


In [159]:
p_con = df_con['Payments'].sum()/df_con['Enrollments'].sum()
p_exp = df_exp['Payments'].sum()/df_exp['Enrollments'].sum()

In [160]:
p_con, p_exp

(0.537120211360634, 0.5682150160677768)